# Get `gasUsed` statistics from previous successful Uniswap V3 pool creations

This notebook shows some basic Python code to query Etherscan's API (needs an API key) for Uniswap V3 pool creation transactions, filter out the successful ones, and extract their `gasUsed` values to get some statistics (mean value, quartiles, etc).

This uses `requests` to query the Etherscan API endpoint, and `pandas` to extract the statistics (percentile values).

(Powered by Etherscan.io APIs)

In [ ]:
# URL endpoint for Etherscan API
uapi = r'https://api.etherscan.io/api'

# Etherscan API key
eskey = 'PasteYourEtherscanAPIkeyHere'

# Uniswap V3 Factory smart contract address
v3fac = '0x1f98431c8ad98523631ae4a59f267346ea31f984'

In [ ]:
import requests

In [ ]:
# Build the API query. Note that Uniswap v3 pool creation tx are "internal tx"
query = {'module': 'account', 'action': 'txlistinternal', 'address': v3fac, 'sort': 'asc', 'apikey': eskey}

In [ ]:
response = requests.get(uapi, params=query)
print(response)

<Response [200]>


In [ ]:
# Extract list of V3 pool contract creation tx
clist = response.json()['result']
len(clist)

1381

In [ ]:
# Filter only the successful pool creation tx
csuccess = [c for c in clist if c['isError'] == '0']
len(csuccess)

1189

In [ ]:
import time

# function to get gasUsed from a tx hash
def getGasUsed(txhash):
    query = {'module': 'proxy', 'action': 'eth_getTransactionReceipt', 'txhash': txhash, 'apikey': eskey}
    resp = requests.get(uapi, params=query)
    # time.sleep(0.1) # Etherscan free API rate = 5 calls per second. If necessary, can hack a delay in here
    gasUsed = int(resp.json()['result']['gasUsed'], 16) # convert hex to int
    return gasUsed

In [ ]:
gases = [getGasUsed(c['hash']) for c in csuccess]
len(gases)

1189

In [ ]:
from pandas import DataFrame

df = DataFrame(gases, columns=['gasUsed'])

df.gasUsed.min()

4557500

In [ ]:
df.gasUsed.quantile([0.05,0.25,0.5,0.75,0.95])

0.05    5116549.2
0.25    5164104.0
0.50    5185631.0
0.75    5246029.0
0.95    5323256.6
Name: gasUsed, dtype: float64

In [ ]:
df.gasUsed.max()

5646304